# Surrogate Illustration
This file illustrates how to use the surrogate model to predict the performance of a set of configurations given by the configurator.

## Download the Files
Download the required model files with
```
curl https://www.automl.org/wp-content/uploads/2019/10/cplex_regions200.zip -O
unzip cplex_regions200.zip
```

## Steps overview
1. `configuration` and `instance` comes from the configurator, it is hardcoded in this illustration. 
1. Load the instances features from the json file.
1. Load the prediction model from the binary file
1. Convert configuration a vector with magic found in the EPM library.
1. Append instance features to the end of the configuration vector into `x`
1. Predict the performance of `x`.

N.B. I did not use the wrapper because that appears to be a collection of hacks to workaround the extremely messy code in the EPM library. 

In [1]:
configurations = {'barrier_algorithm': '0',
 'barrier_crossover': '0',
 'barrier_limits_corrections': '-1',
 'barrier_limits_growth': '1000000000000.0',
 'barrier_ordering': '0',
 'barrier_startalg': '1',
 'emphasis_memory': 'no',
 'emphasis_mip': '0',
 'emphasis_numerical': 'no',
 'feasopt_mode': '0',
 'lpmethod': '0',
 'mip_cuts_cliques': '0',
 'mip_cuts_covers': '0',
 'mip_cuts_disjunctive': '0',
 'mip_cuts_flowcovers': '0',
 'mip_cuts_gomory': '0',
 'mip_cuts_gubcovers': '0',
 'mip_cuts_implied': '0',
 'mip_cuts_mcfcut': '0',
 'mip_cuts_mircut': '0',
 'mip_cuts_pathcut': '0',
 'mip_cuts_zerohalfcut': '0',
 'mip_limits_aggforcut': '3',
 'mip_limits_cutpasses': '0',
 'mip_limits_cutsfactor': '4.0',
 'mip_limits_gomorycand': '200',
 'mip_limits_gomorypass': '0',
 'mip_limits_submipnodelim': '500',
 'mip_ordertype': '0',
 'mip_strategy_backtrack': '0.9999',
 'mip_strategy_bbinterval': '7',
 'mip_strategy_branch': '0',
 'mip_strategy_dive': '0',
 'mip_strategy_file': '1',
 'mip_strategy_fpheur': '0',
 'mip_strategy_heuristicfreq': '0',
 'mip_strategy_lbheur': 'no',
 'mip_strategy_nodeselect': '1',
 'mip_strategy_presolvenode': '0',
 'mip_strategy_probe': '0',
 'mip_strategy_rinsheur': '0',
 'mip_strategy_search': '0',
 'mip_strategy_startalgorithm': '0',
 'mip_strategy_subalgorithm': '0',
 'mip_strategy_variableselect': '0',
 'network_netfind': '2',
 'network_pricing': '0',
 'preprocessing_aggregator': '-1',
 'preprocessing_boundstrength': '-1',
 'preprocessing_coeffreduce': '2',
 'preprocessing_dependency': '-1',
 'preprocessing_dual': '0',
 'preprocessing_fill': '10',
 'preprocessing_linear': '1',
 'preprocessing_numpass': '-1',
 'preprocessing_reduce': '3',
 'preprocessing_relax': '-1',
 'preprocessing_repeatpresolve': '-1',
 'preprocessing_symmetry': '-1',
 'read_scale': '0',
 'sifting_algorithm': '0',
 'simplex_crash': '1',
 'simplex_dgradient': '0',
 'simplex_limits_perturbation': '0',
 'simplex_limits_singularity': '10',
 'simplex_perturbation_switch': 'no',
 'simplex_pgradient': '0',
 'simplex_pricing': '0',
 'simplex_refactor': '0',
 'simplex_tolerances_markowitz': '0.01'}

In [2]:
instance = "instances/mip/data/Regions200/CATS-d_regions-goods200-bids1000/CATS-d_regions-goods200-bids1000_0000.lp"

In [3]:
# Load the list of instance features
# 
import json
with open('./target_algorithms/surrogate/cplex_regions200/inst_feat_dict.cplex_regions200.par10.random.json') as f:
    instances_features = json.load(f)

In [4]:
# Let instance_feature 
# N.B I am not sure why the arrays in the json file are in this strange ndarray serialized form instead of just an plain json array. In any case, I decide not to change the json file to maintain compatibility with aclib.
instance_feature = instances_features[instance]['__ndarray__']

In [5]:
# Load the random forest model
from pyrfr import regression
model = regression.binary_rss_forest()
model.load_from_binary_file('./target_algorithms/surrogate/cplex_regions200/pyrfr_model.cplex_regions200.par10.random.bin')

In [6]:
# Load the configuration space
from ConfigSpace.read_and_write import pcs
with open('./target_algorithms/surrogate/cplex_regions200/config_space.cplex_regions200.par10.random.pcs') as f:
    configurationSpace = pcs.read(f)

In [7]:
# Workaround in jupyter notebook in order to import relative python module. https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('.')

In [8]:
from surrogate import convert_params_to_vec
encoded_configurations = convert_params_to_vec(configurations, configurationSpace)

In [9]:
# Construct the input vector by appending features after configurations
import numpy as np
x = np.hstack([encoded_configurations, instance_feature])

In [10]:
# Predict the performance
model.predict(np.hstack([instance_feature, x]))

1.7902174173194199